In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [2]:
Keyword = pd.read_csv('./Keyword.csv')
Keyword

,0
0,서울특별시 강남구 삼성로85길 33 카페디퍼
1,서울특별시 강남구 남부순환로 2753 롯데시네마 도곡
2,서울특별시 강남구 도산대로1길 10 이소젠 네타
3,서울특별시 강남구 언주로150길 35 카페 베이직먼트(cafe basicment)
4,서울특별시 강남구 봉은사로 502 클로리스 삼성중앙역점
...,...
1702,서울특별시 강남구 학동로 330 서오릉피자 강남논현점
1703,서울특별시 강남구 압구정로29길 71 핑크레이디(Pink Lady)
1704,서울특별시 강남구 테헤란로83길 41 어게인
1705,서울특별시 강남구 개포로 264 꽈알라 보틀숍


In [3]:
# 창이 나오지 않게 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'chromedriver.exe' # 크롬드라이버 경로 설정

source_url = "https://map.kakao.com/" # 리뷰주소 크롤링을 해 올 주소 설정

page_urls = [] #리뷰주소 크롤링 결과를 담을 리스트

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for k in tqdm_notebook(Keyword['0']):
    try:
        driver.get(source_url)

        # 검색창 찾기
        search_bar = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
        search_bar.send_keys(k)
        
        # 검색창 찾기
        searchbutton = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.submit")
        driver.execute_script('arguments[0].click();', searchbutton)

        driver.implicitly_wait(10)

        # 상세보기에 링크된 리뷰주소 가져오기
        moreview = driver.find_element(By.CLASS_NAME, 'moreview')
        page_url = moreview.get_attribute("href")
        page_urls.append(page_url)
    except:
        pass

driver.quit()

page_urls

  0%|          | 0/1707 [00:00<?, ?it/s]

['https://place.map.kakao.com/637651089',
 'https://place.map.kakao.com/1913617889',
 'https://place.map.kakao.com/1688463314',
 'https://place.map.kakao.com/1521663828',
 'https://place.map.kakao.com/1240392567',
 'https://place.map.kakao.com/217560346',
 'https://place.map.kakao.com/969107273',
 'https://place.map.kakao.com/26835380',
 'https://place.map.kakao.com/1512941658',
 'https://place.map.kakao.com/1503763309',
 'https://place.map.kakao.com/1725277896',
 'https://place.map.kakao.com/1072619186',
 'https://place.map.kakao.com/257769033',
 'https://place.map.kakao.com/27413523',
 'https://place.map.kakao.com/7833054',
 'https://place.map.kakao.com/1270729223',
 'https://place.map.kakao.com/7832163',
 'https://place.map.kakao.com/885783935',
 'https://place.map.kakao.com/1961667666',
 'https://place.map.kakao.com/756089106',
 'https://place.map.kakao.com/1017440885',
 'https://place.map.kakao.com/202737056',
 'https://place.map.kakao.com/262071421',
 'https://place.map.kakao.com

In [4]:
# 이미 리뷰주소가 있을 경우 사용
'''page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver' '''

"page_urls = []\n\nfor i in names_df.리뷰주소:\n    page_urls.append(i)\n\noptions = webdriver.ChromeOptions()\noptions.add_argument('headless')\n\npath = 'C:/Users/USER/TIL/chromedriver' "

In [5]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        driver.implicitly_wait(10)

        name = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2')
        name = name.text
        star = driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(1) > div.star_info > div > span > span')
        star = ''.join([i for i in star.get_attribute("style") if i.isdigit()])
        review = driver.find_elements(By.CLASS_NAME, 'txt_comment')
        for i in review:
            review_df.loc[len(review_df)] = [name, star, i.text]

    except:
        # 리뷰가 없거나 비공개인 경우
        pass
    
driver.quit()

review_df

  0%|          | 0/796 [00:00<?, ?it/s]

,name,score,review
0,카페디퍼,80,주차도 할겸 따뜻한라떼 한잔 마시는데 중년의 사장님이 샷이 잘못되었다며 다시 가져다...
1,롯데시네마 도곡,100,역시 새 영화관이라 그런지 시설이 좋습니다\n공기도 쾌적하고 주말에 차가 많은 거 ...
2,롯데시네마 도곡,100,깔끔하네요
3,롯데시네마 도곡,100,"인간적으로 롯데시네마 ""매봉""이라고 하자....\n도곡역 갔다 옴"
4,이소젠네타,100,갠적으로 오복수산보다 훨씬 맛있었음 ..\n진짜 강추
...,...,...,...
1470,원포셰,100,동네 최애 디저트집
1471,바이칼리아,60,공간은 예쁘고 쾌적해서 좋습니다. 지금 정도 날씨에는 야외석을 추천\n디저트랑 음료...
1472,바이칼리아,60,2층 대형카페인데 사람 많지 않고 분위기 좋고 컬러감이 좋아요! 가격은 위치가 위치...
1473,바이칼리아,60,최고예요!! 사장님도 너무 친절하세요😎


In [6]:
# 리뷰 없이 별점만 남긴 리뷰 제거
review_df = review_df[review_df['review'] != '']
review_df

,name,score,review
0,카페디퍼,80,주차도 할겸 따뜻한라떼 한잔 마시는데 중년의 사장님이 샷이 잘못되었다며 다시 가져다...
1,롯데시네마 도곡,100,역시 새 영화관이라 그런지 시설이 좋습니다\n공기도 쾌적하고 주말에 차가 많은 거 ...
2,롯데시네마 도곡,100,깔끔하네요
3,롯데시네마 도곡,100,"인간적으로 롯데시네마 ""매봉""이라고 하자....\n도곡역 갔다 옴"
4,이소젠네타,100,갠적으로 오복수산보다 훨씬 맛있었음 ..\n진짜 강추
...,...,...,...
1470,원포셰,100,동네 최애 디저트집
1471,바이칼리아,60,공간은 예쁘고 쾌적해서 좋습니다. 지금 정도 날씨에는 야외석을 추천\n디저트랑 음료...
1472,바이칼리아,60,2층 대형카페인데 사람 많지 않고 분위기 좋고 컬러감이 좋아요! 가격은 위치가 위치...
1473,바이칼리아,60,최고예요!! 사장님도 너무 친절하세요😎


In [7]:
review_df.to_csv("./review_data.csv", index=False)